# Overview

> You may wish to make a copy of this notebook to avoid overwriting the original.

This drawdown integration connects solutions that impact or are impacted by waste production.  _More about this here_


In [1]:
import sys
sys.path.append('../')   # If you move this notebook to another location, change this path to point to the root directory of the solutions project
import pandas as pd
from pathlib import Path
from integrations import integration_base as base
from integrations import waste_integration as wi

# This is an optional step to make floating point numbers display more compactly.
# If you prefer other options, you can change or delete this without harm.
pd.set_option('display.float_format', lambda x: f"{x:03.5f}".rstrip('0'))

Do **<u>one</u>** of the following two:

In [2]:
# If you are beginning a *new* integration, or starting over

base.integration_clean()
base.integration_start()

In [2]:
# If you are continuing an *existing* integration process

base.integration_start()

# Inputs
<a href='inputs'></a>

We recommend that you browse the contents of file `waste_integration.py`.  The inputs to the integration are in the top section of the file.  The main input that you might wish to modify is the choice of scenarios to use for PDS1, PDS2 or PDS3 for any of the solutions.  Usually you will want to use the scenarios that the solutions themselves have declared to be their "current" set, but if you wanted to replace some, you could do that with something like this:
```python
   wi.composting_scenario_names = ["experimental baseline", "experimental goal", "experimental optimum"]
```
(of course there have to be scenarios with those names in that solution first!)

# Integration Steps

The integration is defined as a series of steps....

In [3]:
wi.integrate()

compost adoption adjusted for 26/141 items by max 461.3709499795839
paper available feedstock adjusted for 141/141 items by max 164.75739145946568
paper consumption adjusted for 52/141 items by max 134.32478104384097
insulation consumption adjusted for 22/141 items by max 17.29156247
wte consumption adjusted for 118/141 items by max 1137.350578099259
lm adoption adjusted for 2/141 items by max 23.53
updating composting adoption
updating recycling adoption
updating waste to energy LHV values
updating landfill methane DOC values


In [2]:
wi.ws_step1()

# Checking out the Intermediate Results

In [4]:
base.show_log()

-----------------------
waste
                     waste tam: (47, 3): [1773.333333 1773.333333 1773.333333]
                  base organic: (47, 3): [804.14197532 804.14197532 804.14197532]
               base recyclable: (47, 3): [627.40533322 627.40533322 627.40533322]
                base remainder: (47, 3): [341.78602447 341.78602447 341.78602447]
     organic less reduced food: (47, 3): [804.14197532 804.14197532 804.14197532]
   organic plus compostable bp: (47, 3): [804.79494532 804.79494532 804.79494532]
 recylable less compostable bp: (47, 3): [626.75236322 626.75236322 626.75236322]
         base compost adoption: (47, 3): [89.275 89.275 89.55 ]
     adjusted compost adoption: (47, 3): [89.275 89.275 89.55 ]
     organics msw less compost: (47, 3): [715.51994532 715.51994532 715.24494532]
       base recycling adoption: (47, 3): [206.02941101 206.02941101 206.02941101]
   adjusted recycling adoption: (47, 3): [206.02941101 206.02941101 206.02941101]
 recyclable msw less recy

In [4]:
df = base.get_logitem("paper consumption")
df.head()

,PDS1,PDS2,PDS3
Year,,,
2014,268.41,268.41,230.88897
2015,278.35576,296.06247,254.6759
2016,288.45107,307.93993,264.89301
2017,298.77627,320.3188,275.54144
2018,309.33687,333.22776,286.64586


<hr/>

### Developer Notes

> The remainder of this notebook is notes of the developer, and can be ignored.

Analysis of formula references:
```
    (PDS1, PDS2, PDS3) <--> (WTE Tonnes & Composition, Paper, Compost, Recycling)
    (PDS1, PDS2, PDS3) <-- Food System Summary
    PDS1 <--> Bioplastics
    (WTE Tonnes & Composition <--> WTE and LM Analysis <--> Landfill Methane)  <-- (PDS1, PDS2, PDS3)
    Refrigerant Management  <-->  None
```
That is, PDS1,2,3 are both input and output for the topic tabs, and the Waste & Landfill Methand tabs are a tightly bound cluster.  Refrigerant Management is isolated, as are all the background sheets, and the MSW TAM Update.

For some reason Bioplastics are tied only with PDS1?

The instruction document is quite detailed and helpful.  We'll mostly follow along.

Later notes:  I started out using openpyxl and SMAs to save things to files, and ended up just manually creating CSV files instead, as it was simpler.

Testing: by creating test snapshots of input data, you can then compare outputs (from the audit log) against the Excel.
I usually do this by cut and paste of Jupyter printout into a spare Excel tab, where I can do side-by-side comparison.

In [1]:
import sys
sys.path.append('../')   # If you move this notebook to another location, change this path to point to the root directory of the solutions project
import pandas as pd
from pathlib import Path
from integrations import integration_base as base
from integrations import waste_integration as wi

# This is an optional step to make floating point numbers display more compactly.
# If you prefer other options, you can change or delete this without harm.
pd.set_option('display.float_format', lambda x: f"{x:03.5f}".rstrip('0'))
base.integration_start(True)

In [2]:
wi.ws_step1()
wi.ws_step2()
wi.ws_step3()
wi.ws_step4()
wi.ws_step5()
wi.ws_step6()
base.show_log()

compost adoption adjusted for 27/141 items by max 264.59547353470407
recycling adjustment adjusted for 21/141 items by max 342.6413576487997
paper available feedstock adjusted for 12/141 items by max 62.30129153091218
wte consumption adjusted for 99/141 items by max 1387.5034609669995
lm adoption adjusted for 42/141 items by max 182.2803018
-----------------------
waste
                     waste tam: (47, 3): [1773.333333 1773.333333 1773.333333]
                  base organic: (47, 3): [804.14197532 804.14197532 804.14197532]
               base recyclable: (47, 3): [627.40533322 627.40533322 627.40533322]
                base remainder: (47, 3): [341.78602447 341.78602447 341.78602447]
     organic less reduced food: (47, 3): [804.14197532 804.14197532 804.14197532]
   organic plus compostable bp: (47, 3): [804.79494532 804.79494532 804.79494532]
 recylable less compostable bp: (47, 3): [626.75236322 626.75236322 626.75236322]
         base compost adoption: (47, 3): [110.5436684 11

In [5]:
base.get_logitem("wte base consumption").head()

,PDS1,PDS2,PDS3
Year,,,
2014,164.5302,164.5302,164.5302
2015,171.47641,171.47641,171.47641
2016,179.36106,179.36106,179.36106
2017,184.87641,184.87641,184.87641
2018,191.24947,191.24947,191.24947


In [6]:
base.integration_clean()